# Photogate explicado paso a paso

## Photogate y MotionDAQ

El [*photogate*](https://www.vernier.com/product/photogate/),
o fotosensor de barrera,
es un sensor que permite medir tiempos de manera muy precisa.
Para ello,
tiene un emisor de luz infrarroja
y un detector de luz.
Cuando detecta la luz infrarroja,
emite una señal de un dado voltaje.
Cuando un objeto atravieza el sensor,
bloquea la luz,
y el sensor emite otro voltaje.
Midiendo el tiempo cuando se produce el cambio de voltaje,
podemos saber cuando un objeto pasó por el *photogate*.

La señal del *photogate* la podemos digitalizar con el MotionDAQ.
Este mide el voltaje que emite el sensor
para un conjunto de tiempos discretos.
El tiempo $\Delta t$ entre mediciones se puede controlar,
elijiendo la frecuencia de muestreo $f = 1/\Delta t$.
Luego,
podemos exportar estas mediciones como un archivo de texto.

En este notebook,
vamos a ver como podemos cargar
y analizar esas mediciones para obtener los tiempos en que se bloqueó o desbloquó el sensor.

## Paquetes

Importamos los paquetes y funciones que vamos a necesitar:

- la función `Path` de `pathlib`,
  que nos permite manejar las "rutas" o ubicaciones de los archivos.
  En particular,
  la vamos a usar para encontrar todos los archivos de medición que usemos.
- `numpy`, para operar con arrays numéricos
- `matplotlib`, para graficar
- `pandas`, que solo vamos a usar para leer los archivos.
  Es muy usado porque proporciona varias comodidades sobre `numpy`,
  pero es confuso de usar al principio.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Buscando los archivos

Buscamos todos los archivos que terminan con `.txt` en la carpeta actual.

:::{note}
Pueden decargar unos archivos de muestra de acá:
- [1cm.txt](https://github.com/maurosilber/python-tutorial/raw/main/book/laboratorio/photogate/1cm.txt)
- [2cm.txt](https://github.com/maurosilber/python-tutorial/raw/main/book/laboratorio/photogate/2cm.txt)
:::

In [ ]:
carpeta = Path()  # carpeta actual
archivos = list(carpeta.glob("*.txt"))  # lista de archivos que terminan con .txt

archivos

## Cargando datos de un archivo

Si abrimos un archivo en un editor de texto (`notepad`),
veriamos lo siguiente:

In [ ]:
!head "1cm.txt"

Es un archivo de texto,
donde las primeras dos filas tienen *metadata*:

- la hora a la que se tomó la medición, y
- número de corrida.

Las siguientes filas tienen los datos
agrupados en dos columnas,
que estan separadas por "un espacio".
En programación,
hay al menos dos tipos de espacios:

- `" "`, el espacio común que inserta la barra espaciadora, y
- `"\t"`, el espacio que inserta la tecla Tab.

En este caso,
es el segundo.

También,
los números usan una coma como separador decimal.

Todo esto hay que decirselo a Python,
para que sepa como interpretar el archivo.

In [ ]:
df = pd.read_csv(
    archivos[0],  # el primer archivo,
    skiprows=3,  # que ignore las primeras 3 filas
    delimiter="\t",  # separador Tab
    decimal=",",  # coma decimal
)

df

La variable `df` tiene un `DataFrame` de `pandas`.
Podemos extraer el `array` de `numpy` interno con `df.values`:

In [ ]:
data = df.values

data

Este `array` tiene 996 filas y 2 columnas:

In [ ]:
data.shape

Para más (o menos) comodidad,
podemos separar cada una en una variable:

In [ ]:
t = data[:, 0]  # tiempo [s]
v = data[:, 1]  # voltaje [V]

## Graficar los datos

Es importante visualizar los datos,
para asegurarnos de que no haya habido problemas de medición.
Grafiquemos estas dos variables:

In [ ]:
plt.figure(figsize=(6, 2))
plt.xlabel("Tiempo [s]")
plt.ylabel("Voltaje [V]")
plt.plot(t, v)

Hagamos zoom sobre el primer salto,
y graficamos los valores discretos de la señal:

In [ ]:
plt.figure(figsize=(6, 2))
plt.xlabel("Tiempo [s]")
plt.ylabel("Voltaje [V]")
plt.plot(t, v, marker="o", linestyle="--")
plt.xlim(0.12, 0.18)
plt.grid()

### Preguntas

- ¿En qué tiempo se bloqueó el sensor?
- ¿Qué error podemos asignarle a ese tiempo?
- ¿Cómo podemos encontar "automaticamente" ese tiempo?
  ¿Qué distingue a esos tiempos de los demás?

## Calcular diferencias de voltaje

Para encontrar el cambio de voltaje,
podemos calcular la diferencia entre un punto $i$ y el siguiente $i+1$.
`numpy` incluye una función para esto: `np.diff`.

In [ ]:
dif_de_v = np.diff(v)

Este vector tiene un valor menos el original,
ya que no puede calcular la diferencia del último con el siguiente
(¡ya no hay siguiente!).

In [ ]:
np.size(v)

In [ ]:
np.size(dif_de_v)

Para graficarlo,
vamos a ignorar el tiempo,
y dejar que grafique contra el número de indice del array:

In [ ]:
plt.figure(figsize=(6, 2))
plt.xlabel("Indice del array")
plt.ylabel("Dif. de voltaje [V]")
plt.plot(dif_de_v, marker=".", linestyle="--")

Si hacemos zoom al primer salto,
y superponemos la señal original,
podemos ver que da (practicamente) 0 en todos lados,
menos en los cambios de voltaje:

In [ ]:
plt.figure(figsize=(6, 2))
plt.xlabel("Indice del array")
plt.ylabel("Voltaje [V]")
plt.plot(v, label="Voltaje", marker=".", linestyle="--")
plt.plot(dif_de_v, label="Dif. de voltaje", marker=".", linestyle="--")
plt.legend()
plt.xlim(0, 40)

Pero ojo,
no es exactamente 0,
ya que el voltaje fluctua levemente:

In [ ]:
v[:5]

In [ ]:
dif_de_v[:5]

## Encontrar valores que cumplen cierta condicion

Para encontrar los valores dentro de un `array` que cumplen cierta condicion,
se pueden hacer lo siguiente:

In [ ]:
y = np.array([0, 0, 5, 5, 0, 5])  # array de ejemplo

y > 3

que devuelve un vector de verdaderos (`True`) y falsos (`False`) para cada elemento.

Si quieren obtener las posiciones o indices donde se cumplió la condición,
es decir, donde están los `True`,
pueden hacer así:

In [ ]:
pos = np.nonzero(y > 3)

pos

Si quisieran extraer los valores de `y` que están en esas posiciones:

In [ ]:
y[pos]

Entonces,
en el caso de la señal del fotosesnsor,
podemos encontrar los "saltos",
tanto positivos y negativos,
como:

In [ ]:
pos = np.nonzero(np.abs(dif_de_v) > 2.5)

pos

Peor no nos interesa obtener el valor del voltaje en esas posiciones,
sino los tiempos en esas posiciones:

In [ ]:
tiempo_saltos = t[pos]

tiempo_saltos

## Encontrar periodos

Finalmente,
si nos interesa encontrar periodos,
podemos calcular las diferencias entre los tiempos correspondientes.
Esto dependerá del experimento.

Por ejemplo,
si necesitamos obtener 1 de cada 4 tiempos,
podemos tomar una "rebanada" (*slice*) del array con la siguiente expresion:

In [ ]:
tiempo_saltos[::4]

En general,
la notación es `array[desde:hasta:paso]`.
Si omitimos uno,
por defecto son:

- `desde`: 0
- `hasta`: hasta el final
- `paso`: 1

Por ejemplo:

- `array[2:8:3]`
- `array[:5]`: hasta el 5to elemento.
- `array[10::3]`: desde el 10mo elemento cada 3.
- etc.

Luego,
podriamos calcular la diferencia entre valores consecutivos con:

In [ ]:
np.diff(tiempo_saltos[::4])

## Más periodos

Para aprovechar todos los datos,
también pueden calcular los periodos cada 4,
pero empezando de la segunda medicion (indice 1):

In [ ]:
np.diff(tiempo_saltos[1::4])

Y desde la tercera y cuarta medicion.

Pueden hacer todo esto en un paso con la siguiente expresion:

In [ ]:
tiempo_saltos[4:] - tiempo_saltos[:-4]

Para ver que esta haciendo,
generemos un array más simple:

In [ ]:
x = np.arange(6)
x

Si tomamos desde el segundo hasta el final,
obtenemos:

In [ ]:
x[2:]

Y si tomamos desde el principio hasta 2 menos del final,
tenemos:

In [ ]:
x[:-2]

Luego,
haciendo la diferencia elemento a elemento,
es hacer la diferencia cada dos elementos:

In [ ]:
x[2:] - x[:-2]